# Data Exploration  


Goal: how many of each event type we have 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import os


from src.utils import *
from src.plotting import Baseball_Field
from src.game import Game

In [2]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'


## See what kind of variables might be nice to have

In [3]:
# "1900_02_TeamKJ_TeamB" is what I have been working with so far,
# but this is early, so it might not have the later events?

which_game = "1903_02_TeamNE_TeamA2" # "1903_01_TeamNE_TeamA2" # "1900_02_TeamKJ_TeamB"

In [4]:
game_info_df = pd.read_csv("data/game_info/game_info-" + which_game + ".csv", index_col=0)
game_info_df.head()

,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
1,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,1,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,4474,0,0,0
2,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,2,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,4474,0,0,0
3,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,3,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,4474,0,0,0
4,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,4,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,0,4474,0,0
5,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,2,5,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,3790,4474,0,0


In [55]:
game_events_df = pd.read_csv("data/game_events/game_events-" + which_game + ".csv", index_col=0)
game_events_df["event"] = game_events_df["event_code"].map(lambda x: EVENT_CODE_TO_DESC[x])
game_events_df.head()

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,event
1,1903_02_TeamNE_TeamA2,1,1,1,46725,1,1,pitch
2,1903_02_TeamNE_TeamA2,1,1,1,47325,10,4,ball hit into play
3,1903_02_TeamNE_TeamA2,1,1,1,47375,0,5,end of play
4,1903_02_TeamNE_TeamA2,2,1,2,60475,1,1,pitch
5,1903_02_TeamNE_TeamA2,2,1,2,61125,2,2,ball acquired


#### All of the plays that have a 0 for batter, but have a ball hit into play

In [56]:
zero_batters = game_info_df.loc[game_info_df["batter"] == 0, :]
zero_batters

,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
4,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,4,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,0,4474,0,0
131,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,133,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,0,1557,1751,0


In [57]:
game_info_df.loc[game_info_df["play_per_game"].isin([3, 4, 5]), :]

,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
3,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,3,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,4474,0,0,0
4,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,1,4,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,0,4474,0,0
5,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,2,5,1,Top,1077,1240,2526,1557,1297,2148,2053,1177,1751,3790,4474,0,0


In [59]:
game_events_df.loc[(game_events_df["play_per_game"].isin(zero_batters["play_per_game"]))# &\
                   #(game_events_df["event_code"] == 4)
                   , ["game_str", "play_id", "play_per_game", "event_code"]]

,game_str,play_id,play_per_game,event_code
10,1903_02_TeamNE_TeamA2,4,4,1
11,1903_02_TeamNE_TeamA2,4,4,4
12,1903_02_TeamNE_TeamA2,4,4,16
13,1903_02_TeamNE_TeamA2,4,4,16
14,1903_02_TeamNE_TeamA2,4,4,16
15,1903_02_TeamNE_TeamA2,4,4,2
16,1903_02_TeamNE_TeamA2,4,4,3
17,1903_02_TeamNE_TeamA2,4,4,2
18,1903_02_TeamNE_TeamA2,4,4,5
492,1903_02_TeamNE_TeamA2,132,133,1


#### find all the instances where a ball is hit into play, but it shows that there is no batter

In [47]:
all_games = []

for sub_dir, dirs, files in os.walk('data/game_events/'):
    for file in files:
        if "checkpoint" in file:
                continue
    
        all_games.append(file.split("-")[-1].split(".csv")[0])

print(len(all_games))        

97


In [62]:
zeros = pd.DataFrame()

for g in all_games:
    this_game = Game(g)
    
    zero_batters = this_game.game_info_df.loc[this_game.game_info_df["batter"] == 0, :]

    events_w_data = this_game.game_events_df.loc[(this_game.game_events_df["play_per_game"].isin(zero_batters["play_per_game"])) # &\
                                                #(this_game.game_events_df["event_code"] == 4)
                                                , ["game_str", "play_id", "play_per_game", "event_code"]]
    
    zeros = pd.concat([zeros, events_w_data])
    


In [65]:
zeros[["game_str", "play_id", "play_per_game"]].drop_duplicates()

,game_str,play_id,play_per_game
377,1903_01_TeamNE_TeamA2,101,101
622,1903_01_TeamNE_TeamA2,160,160
625,1903_01_TeamNE_TeamA2,161,161
628,1903_01_TeamNE_TeamA2,162,162
774,1903_01_TeamNE_TeamA2,203,203
445,1903_11_TeamNC_TeamA1,110,111
10,1903_02_TeamNE_TeamA2,4,4
492,1903_02_TeamNE_TeamA2,132,133
520,1903_25_TeamNK_TeamB,126,131
567,1903_28_TeamNF_TeamA2,158,158


In [72]:
zeros[["game_str", "play_id", "play_per_game"]].drop_duplicates().to_csv("data_debugging/batter_as_zeros.csv", index = False)

In [66]:
game_1903_01 = Game("1903_01_TeamNE_TeamA2")

In [70]:
game_1903_01.game_info_df.loc[(game_1903_01.game_info_df["batter"]) == 0, :]

,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
101,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,25,101,3,Bottom,8523,7071,5616,6993,9762,6000,3790,4474,7148,0,2053,0,0
159,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,42,160,5,Top,2171,2790,2526,2053,2382,1972,1557,1751,2973,0,6000,0,0
160,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,42,161,5,Top,2171,2790,2526,2053,2382,1972,1557,1751,2973,0,6000,0,0
161,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,42,162,5,Top,2171,2790,2526,2053,2382,1972,1557,1751,2973,0,6000,0,0
202,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,51,203,6,Top,2171,2790,2526,2053,2382,1972,1557,1751,2973,0,0,6803,9762


In [ ]:
game_1903_01.game_info_df.loc[(game_1903_01.game_info_df["batter"]) == 0, :]

In [ ]:
game_events_monitoring = pd.DataFrame()

root_dirs = ['data/game_events/'] 


for root_dir in root_dirs:
    for sub_dir, dirs, files in os.walk(root_dir):
        for file in files:
            if "checkpoint" in file:
                continue
            
            temp_game_events_df = pd.read_csv(os.path.join(sub_dir, file), index_col=0)
            temp_game_events_df["event"] = temp_game_events_df["event_code"].map(lambda x: EVENT_CODE_TO_DESC[x])
            
            temp_df = pd.DataFrame(temp_game_events_df["event"].value_counts(normalize=True))
            
            # take just the first part
            temp_df["game"] = file.split(".csv")[0]
            
            temp_df = temp_df.reset_index(drop=False)
            
            game_events_monitoring = pd.concat([game_events_monitoring, temp_df])

            
game_events_monitoring = game_events_monitoring.reset_index()                


#### Do the fields I except to line up, actually line up?

In [6]:
mismatched_play_ids = game_events_df.loc[game_events_df["play_id"] != game_events_df["play_per_game"], :]

In [7]:
mismatched_play_ids

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,event
480,1903_02_TeamNE_TeamA2,128,46,129,5175437,1,1,pitch
481,1903_02_TeamNE_TeamA2,128,46,129,5175987,2,2,ball acquired
482,1903_02_TeamNE_TeamA2,128,46,129,5175987,0,5,end of play
483,1903_02_TeamNE_TeamA2,129,46,130,5204387,1,1,pitch
484,1903_02_TeamNE_TeamA2,129,46,130,5204837,2,2,ball acquired
...,...,...,...,...,...,...,...,...
841,1903_02_TeamNE_TeamA2,211,69,222,8295387,10,4,ball hit into play
842,1903_02_TeamNE_TeamA2,211,69,222,8295437,0,5,end of play
843,1903_02_TeamNE_TeamA2,211,69,222,8326187,1,1,pitch
844,1903_02_TeamNE_TeamA2,211,69,222,8326687,2,2,ball acquired


In [15]:
#mismatched_play_ids["play_id_shifted"] = 
first_mismatch = mismatched_play_ids.loc[mismatched_play_ids["play_id"].diff() != mismatched_play_ids["play_per_game"].diff(), :]
first_mismatch

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,event
480,1903_02_TeamNE_TeamA2,128,46,129,5175437,1,1,pitch
812,1903_02_TeamNE_TeamA2,211,69,222,8050687,1,1,pitch


In [21]:
# Something weird is going on with play id 

# HMM, play_per_game just skips? I wonder if that was like a pitching change or something

game_events_df.loc[(game_events_df["play_id"] >= first_mismatch.iloc[0]["play_id"] - 3) & (game_events_df["play_id"] <= first_mismatch.iloc[0]["play_id"] + 3) , :]

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,event
471,1903_02_TeamNE_TeamA2,125,46,125,5084637,1,1,pitch
472,1903_02_TeamNE_TeamA2,125,46,125,5085187,2,2,ball acquired
473,1903_02_TeamNE_TeamA2,125,46,125,5085187,0,5,end of play
474,1903_02_TeamNE_TeamA2,126,46,126,5102437,1,6,pickoff throw
475,1903_02_TeamNE_TeamA2,126,46,126,5103037,3,2,ball acquired
476,1903_02_TeamNE_TeamA2,126,46,126,5103037,0,5,end of play
477,1903_02_TeamNE_TeamA2,127,46,127,5123287,1,1,pitch
478,1903_02_TeamNE_TeamA2,127,46,127,5123787,2,2,ball acquired
479,1903_02_TeamNE_TeamA2,127,46,127,5123787,0,5,end of play
480,1903_02_TeamNE_TeamA2,128,46,129,5175437,1,1,pitch


In [26]:
game_info_df.loc[(game_info_df["play_per_game"] >= 125) & (game_info_df["play_per_game"] <= 130), :]

,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
123,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,125,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0
124,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,126,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0
125,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,127,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0
126,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,128,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0
127,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,129,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0
128,1903_02_TeamNE_TeamA2,TeamA2,TeamNE,46,130,5,Bottom,4619,7071,6803,4306,5616,6000,3790,4474,7148,1557,1751,0,0


In [ ]:
mismatched_play_ids["play_id_shifted"] =  mismatched_play_ids["play_id"].shift()

mismatched_play_ids["play_per_game"] =  mismatched_play_ids["play_per_game"].shift()


In [ ]:
mismatched_play_ids.loc[mismatched_play_ids["play_id"] != mismatched_play_ids["play_id_shifted"] ]

## What I notice isn't there that might be nice to have:
- the at_bat field is not there for older games
- I don't think I have any outs or count
    - Not sure I can get counts
- ~~Players speed at a given time~~
- Player orientation?
    - I think I could noisily infer this
- I don't think that "ball bounce" is in the older data -- how is this defined?
    - I could maybe fill this one in if z_coord is < 0.5 or something arbitrarily small?

### Try to fill in the at_bat field

In [ ]:
ball_hitting_something_df = game_events_df.loc[game_events_df["event"].isin(["ball deflection", "ball deflection off of wall", "ball bounce"]), :]

In [ ]:
ball_hitting_something_df["event"].value_counts()

In [ ]:
def add_at_bat_data(df): 
    pass

In [ ]:
game_info_df.head()

In [ ]:
# assumption: an at_bat is when the pitcher and batter are the same?
# this is wrong when a pitcher is changed mid inning
# this is also wrong when a batter is changed 

game_info_df.loc[:5, ["inning", "top_bottom_inning", "pitcher", "batter",]].groupby(["inning", "top_bottom_inning", "pitcher", "batter"]).cumcount() + 1

In [ ]:
grouping_cols = ["inning", "top_bottom_inning", "pitcher", "batter"]

new_game_info_df = game_info_df.copy()


new_game_info_df.loc[:10, grouping_cols]

In [ ]:
new_game_info_df["play_per_pa"] = new_game_info_df.loc[:, :].groupby(grouping_cols).cumcount() + 1

In [ ]:
# this is a check: the play_per_game should be equal to the running sum of play_per_pa when there is a new at bat?

new_game_info_df["play_per_pa"].head(10).cumsum()

In [ ]:
new_game_info_df.head()

In [ ]:
new_game_info_df["new_at_bat"] = (new_game_info_df.loc[new_game_info_df["play_per_pa"] == 1 , ["play_per_pa"]]\
.groupby("play_per_pa").cumcount() + 1)

new_game_info_df["new_at_bat"] = new_game_info_df["new_at_bat"].fillna(method='ffill', inplace=False).astype(int)

In [ ]:
new_game_info_df.loc[new_game_info_df["at_bat"] + 1 == new_game_info_df["new_at_bat"], :].head()

In [ ]:
new_game_info_df["new_at_bat_shifted"] = new_game_info_df["new_at_bat"].shift()

In [ ]:
new_game_info_df.loc[(new_game_info_df["at_bat"] != new_game_info_df["new_at_bat"]) &\
                     (new_game_info_df["new_at_bat"] != new_game_info_df["new_at_bat_shifted"])]

In [ ]:
new_game_info_df.loc[(new_game_info_df["at_bat"] != new_game_info_df["new_at_bat"]) & (new_game_info_df["batter"] == 0), :]

In [ ]:
new_game_info_df.loc[130:135, :]

In [ ]:
# WHAT THE HECK, THE PLAY_PER_GAME AND PLAY_ID ARE DIFFERENT?

In [ ]:
# HMM I kind of suspect that this batter being 0 would mean that the batter for the next play is the correct batter

In [ ]:
game_events_df.loc[game_events_df["play_id"] == 132, :]

In [ ]:
game_events_df.loc[game_events_df["play_id"] == 133, :]

In [ ]:
player_pos = bf.game_obj.player_pos_df.copy()

In [ ]:
# I think I expect this one to be 2053

# Also batters are at a frequency of 20 Hz it looks like?

player_pos.loc[(player_pos["play_id"] == 133 ) & ( player_pos["player_position"] == 10), :]

In [ ]:
# Something is really off here about how I am calculating ABs

bf = Baseball_Field(which_game=which_game)


In [ ]:
bf.create_gif(play_id=3, tag="_at_bat_probs")

In [ ]:
bf.create_gif(play_id=4, tag="_at_bat_probs")

In [ ]:
bf.create_gif(play_id=133, tag="_at_bat_probs")

In [ ]:
## something happens at play 101, where a batter is awarded first with no batter? I think this is a IBB or HBP?

new_game_info_df.loc[98:105, :]

In [ ]:
game_events_df.loc[game_events_df["play_per_game"] == 101, :]

In [ ]:
new_game_info_df.loc[new_game_info_df["at_bat"] + 2 == new_game_info_df["new_at_bat"], :]

In [ ]:
# Something weird also happens at play id 160 and 202

new_game_info_df.loc[158:162, :]

In [ ]:
# gotta be honest, I am a bit confused by this one -- there are several plays in a row, where the 
# events show a pitch, but the game info doesn't show a batter?
game_events_df.loc[game_events_df["play_per_game"] == 162, :]

In [ ]:
game_info_df.loc[200:205, :]

In [ ]:
new_game_info_df.loc[200:205, :]

In [ ]:
game_events_df.loc[game_events_df["play_per_game"] == 202, :]

In [ ]:
# I am a bit confused by this one as well -- this one has a play which shows the batter and runner on first both advance?
# so is that like a dropped 3rd that kicks around the backstop?

game_events_df.loc[game_events_df["play_per_game"] == 203, :]

In [ ]:
# gotta be honest, I am a bit confused by this one -- there are several plays in a row, where the 
# events show a pitch, but the game info doesn't show a batter?
game_events_df.loc[game_events_df["play_per_game"] == 203, :]

In [ ]:
new_game_info_df.loc[new_game_info_df["at_bat"] + 3 == new_game_info_df["new_at_bat"], :]

#### Use at_bat field to fill in outs?

- double check me on that

In [ ]:
### 

### What are the most common events?
- Corollary: when were they added?

In [ ]:
# These figures aren't all that useful as they stand honestly

In [ ]:
game_events_monitoring = pd.DataFrame()

root_dirs = ['data/game_events/'] 


for root_dir in root_dirs:
    for sub_dir, dirs, files in os.walk(root_dir):
        for file in files:
            if "checkpoint" in file:
                continue
            
            temp_game_events_df = pd.read_csv(os.path.join(sub_dir, file), index_col=0)
            temp_game_events_df["event"] = temp_game_events_df["event_code"].map(lambda x: EVENT_CODE_TO_DESC[x])
            
            temp_df = pd.DataFrame(temp_game_events_df["event"].value_counts(normalize=True))
            
            # take just the first part
            temp_df["game"] = file.split(".csv")[0]
            
            temp_df = temp_df.reset_index(drop=False)
            
            game_events_monitoring = pd.concat([game_events_monitoring, temp_df])

            
game_events_monitoring = game_events_monitoring.reset_index()                


In [ ]:
# some plots by game might be useful here, or like a little dashboard or something 

In [ ]:
game_events_monitoring

In [ ]:
game_events_monitoring_df = game_events_monitoring.pivot(index='game', columns="index", values="event").sort_index()

In [ ]:
game_events_monitoring_df = game_events_monitoring_df.reset_index()

game_events_monitoring_df["away"] = game_events_monitoring_df["game"].str.split("_Team").apply(lambda x: x[1])
game_events_monitoring_df["home"] = game_events_monitoring_df["game"].str.split("_Team").apply(lambda x: x[-1])

game_events_monitoring_df["year"] = game_events_monitoring_df["game"].str.strip("game_events-").str.split("_").apply(lambda x: x[0])
game_events_monitoring_df["game_num"] = game_events_monitoring_df["game"].str.strip("game_events-").str.split("_").apply(lambda x: x[1])

# game_events_monitoring_df = game_events_monitoring_df.set_index('game')

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12,12))

i =0
for team in ['B', 'A3', 'A2', 'A1']:
    
    lines = game_events_monitoring_df.loc[game_events_monitoring_df["home"] == team, :].plot(kind = "line", ax=ax[i // 2][i % 2])
    
    i += 1

plt.xticks(rotation=90)

In [ ]:
for year in ["1900", "1901", "1902", "1903"]:
    i = 0
    
    this_year = game_events_monitoring_df.loc[(game_events_monitoring_df["year"] == year), :]
    this_year_teams = this_year["home"].unique()
    
    fig, ax = plt.subplots(nrows=4 , figsize=(12,12))

    for team in this_year["home"].unique():

        plot_df = game_events_monitoring_df.loc[(game_events_monitoring_df["home"] == team) & (game_events_monitoring_df["year"] == year), :]
        
        
        plot_df.plot(kind = "line", x="game_num", ax=ax[i])

        i += 1

plt.xticks(rotation=90)

### How many throws does each player(/position) make?

In [ ]:
all_game_events = concat_game_events_into_single_file(write=False)

In [ ]:
all_game_events.head()

In [ ]:
all_game_events["event"].value_counts()

In [ ]:
group_by_cols = ["player_position", "event"]


first_base_receives = all_game_events.loc[(all_game_events["event"] == "ball acquired") &\
                                      (all_game_events["player_position"] == 3)&\
                                      (all_game_events["last_event"] == "throw (ball-in-play)"), :]

first_base_receives.loc[:, "last_player_position"].value_counts(normalize = True).plot(kind="barh", title="First Baseman Recieves across all data")

plt.ylabel("Throwing Player")
plt.xlabel("Frequency")

print(first_base_receives.shape)



In [ ]:
game_events_df["last_event_code"] = game_events_df["event_code"].shift(1)

In [ ]:
game_events_df["last_player_position"] = game_events_df["player_position"].shift(1)

In [ ]:
game_events_df["last_event"] = game_events_df["last_event_code"].map(lambda x: EVENT_CODE_TO_DESC[x] if pd.notnull(x) else x)

In [ ]:
this_game_first_base_acqs = game_events_df.loc[(game_events_df["event"] == "ball acquired") &\
                                               (game_events_df["player_position"] == 3) &\
                                               (game_events_df["last_event"] == "throw (ball-in-play)")
                                               , :]

In [ ]:
this_game_first_base_acqs.shape[0]

### How often is the ball acquired by each player?